In [1]:
# Import
import MetaTrader5 as mt5
import logging

from pandas import read_csv
from datetime import datetime, timedelta
import matplotlib as mpl
from matplotlib import pylab as plt
import pandas as pd

import numpy as np
import tensorflow as tf

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
import pickle
from keras import backend as K

Using TensorFlow backend.


In [3]:
d = datetime.now()
dstring = str(d.day) + '-' + str(d.hour) + '-' + str(d.minute)
logging.basicConfig(filename='ForEx-Log-' + dstring,
                            filemode='a',
                            format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                            datefmt='%H:%M:%S',
                            level=logging.DEBUG)

logging.info("Running FOREX TRADER" + dstring)
directory = "C:/Users/jonat/Desktop/Picsou/trader/"


AttributeError: 'NoneType' object has no attribute 'point'

In [4]:
import MetaTrader5 as mt5
# display data on the MetaTrader 5 package
print("MetaTrader5 package author: ",mt5.__author__)
print("MetaTrader5 package version: ",mt5.__version__)
 
# establish connection to the MetaTrader 5 terminal
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()
  
# get symbols containing RU in their names
ru_symbols=mt5.symbols_get("*EUR*")
print('len(*EUR*): ', len(ru_symbols))
for s in ru_symbols:
    print(s.name)
print()

point = mt5.symbol_info('EURUSD').point
print(point)
point = mt5.symbol_info('EURZAR').point
print(point)
 
# shut down connection to the MetaTrader 5 terminal
mt5.shutdown()

MetaTrader5 package author:  MetaQuotes Software Corp.
MetaTrader5 package version:  5.0.33
len(*EUR*):  19
EURUSD
EURAUD
EURCAD
EURCHF
EURCZK
EURDKK
EURGBP
EURHKD
EURHUF
EURJPY
EURNOK
EURNZD
EURPLN
EURRUR
EURRUB
EURSEK
EURTRY
EURZAR
EURMXN

1e-05
1e-05


True

In [24]:
#connect
server = 'MetaQuotes-Demo'
password = 'cpe1ppwr'
account = 33050808
TIMEOUT = 600000

# establish connection to the MetaTrader 5 terminal
if not mt5.initialize():
    logging.info("initialize() failed, error code ="+mt5.last_error())
    quit()
    

authorized=mt5.login(account, password=password, server=server, timeout=TIMEOUT)
if authorized:
    
    # display trading account data in the form of a list
    logging.info("Show account_info()._asdict():")
    account_info_dict = mt5.account_info()._asdict()
    for prop in account_info_dict:
        logging.info("  {}={}".format(prop, account_info_dict[prop]))
else:
    logging.info("failed to connect at account #{}, error code: {}".format(account, mt5.last_error()))
 


In [4]:
# get currencies rates
currencies = ['EURCAD', 'EURCHF', 'EURGBP', 'EURJPY', 'EURUSD', 'EURCZK', 'EURDKK', 'EURHKD', 'EURHUF', 'EURMXN', 'EURNOK', 'EURNZD', 'EURPLN', 'EURRUB', 'EURSEK', 'EURTRY', 'EURZAR']
#currencies = ['EURCAD']

lookback = 12
date = datetime.now() + timedelta(hours=2)
date = date.replace(second=0, microsecond=0, minute=0)
logging.info(date)
last_rates = {}

for curr in currencies:
    curr_rates = []
    curr_info = mt5.copy_rates_from(curr, mt5.TIMEFRAME_H1, date, lookback)
    #print(curr_info)
    for v in curr_info:
        logging.info(curr+ ' h: '+ str(datetime.fromtimestamp(v[0]))+ ' rates '+ str(v[4]))
        curr_rates.append(v[4])
        
    last_rates[curr] = curr_rates


In [5]:
def prepareRates(last_rates, scalers):
    data_curr = {}
    for curr, val in last_rates.items():
        
        data = {}
        
        scaler = scalers[curr]        
        dataset = np.reshape(val, (1, len(val)))
        scaled_data = scaler.transform(dataset)
                
        data['dataset'] = scaled_data
        data['scaler'] = scaler
        
        data_curr[curr] = data
                
    return data_curr

In [6]:
def predictRates(data_curr, models):
    
    predictions = {}

    i = 0
    for curr, model in models.items():
        x_test = data_curr[curr]['dataset']
        X = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)
        
        prediction = model.predict(X)
        predictions[curr] = prediction.reshape(prediction.shape[0], 1)
        i += 1
    return predictions

In [7]:
with open(directory + 'scalers', 'rb') as handle:
    scalers = pickle.load(handle)
    
data_curr = prepareRates(last_rates, scalers)

In [8]:
def loadModel(curr_key):
    i = 0
    models = {}
    for curr in curr_key:
        # load the model from disk
        filename = directory + curr + '_model.sav'
        loaded_model = pickle.load(open(filename, 'rb'))
        models[curr] = loaded_model
    return models

In [9]:
models = loadModel(currencies)
predictions = predictRates(data_curr, models)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



In [10]:
logging.info(predictions)

In [11]:
def getActions(data_curr, predictions):

    A = {}
    test_score = 0
    
    for curr, dataset in data_curr.items():        
        action = ''
        
        y = dataset['dataset']
        n = len(y)
        d_cur = y[0][-1] - y[0][-2]

        d_next = predictions[curr][0][0] - y[0][-1]
        
        if d_cur > 0 and d_next < 0:
            action = 'BUY'

        elif d_cur < 0 and d_next > 0:
            action = 'SELL'
        A[curr] = action
    
    return A

In [12]:
A = getActions(data_curr, predictions)
logging.info(str(A))

In [25]:
def get_info(symbol):
    '''https://www.mql5.com/en/docs/integration/python_metatrader5/mt5symbolinfo_py
    '''
    # get symbol properties
    info=mt5.symbol_info(symbol)
    return info

def open_trade(action, Cash, symbol, sl_points, tp_points, deviation):
    '''https://www.mql5.com/en/docs/integration/python_metatrader5/mt5ordersend_py
    '''
    # prepare the buy request structure
    ea_magic_number = 234000
    symbol_info = get_info(symbol)
    
    if action == 'BUY':
        trade_type = mt5.ORDER_TYPE_BUY
        price = mt5.symbol_info_tick(symbol).ask
    elif action =='SELL':
        trade_type = mt5.ORDER_TYPE_SELL
        price = mt5.symbol_info_tick(symbol).bid
                
    point = mt5.symbol_info(symbol).point
    leverage = 100
    tradeVolume = Cash * price * leverage / ( 100000)
    lot = round(tradeVolume, 2)
    lot = 0.01
    
    profit = 1.0
    leverage = 500
    ref = 220
    
    #tp = price + price*profit/(lot*100000) # calculate tp for exact profit € depending on lot and price
    topprice = price/(1-profit/(lot*100000)) # calculate tp for exact profit € depending on lot and price
    tp = round(topprice, 5)
    
    print('Point', point)

    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": trade_type,
        "price": price,
        "deviation": deviation,
        "tp": tp,
        "magic": ea_magic_number,
        "comment": "sent by python",
        "type_time": mt5.ORDER_TIME_GTC, # good till cancelled
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    # send a trading request
    if lot >= mt5.symbol_info(symbol).volume_min:
        result = mt5.order_send(request)
    else:
        result = 'lot = ' + str(tradeVolume)
    return result, request 

def close_trade(position_id, symbol, lot, deviation):
    '''https://www.mql5.com/en/docs/integration/python_metatrader5/mt5ordersend_py
    '''
    # create a close request
    action = 'SELL'
    ea_magic_number = 234000
    if action == 'BUY':
        trade_type = mt5.ORDER_TYPE_BUY
        price = mt5.symbol_info_tick(symbol).ask
    elif action =='SELL':
        trade_type = mt5.ORDER_TYPE_SELL
    price = mt5.symbol_info_tick(symbol).bid
        
    close_request={
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_SELL,
        "position": position_id,
        "price": price,
        "deviation": deviation,
        "magic": ea_magic_number,
        "comment": "python script close",
        "type_time": mt5.ORDER_TIME_GTC, # good till cancelled
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    # send a close request
    result=mt5.order_send(close_request)
    return result, close_request 

In [26]:
sl_points = 100
tp_points = 1000
deviation = 50
count_buy = 0
for curr, a in A.items():
    if a == 'BUY':
        count_buy += 1

In [27]:
for symbol, action in A.items():
    
    logging.info(action + ' ' + symbol)
    
    proft_margin = 0.1
    positions=mt5.positions_get(symbol=symbol)
    for position in positions:
        if position.profit > proft_margin:
            lot = position.volume
            position_id = position.ticket
            result, close_request  = close_trade(position_id, symbol, lot, deviation)
            logging.info('result:'+ str(result))
            logging.info('close_request:'+ str(close_request))

    if action == 'SELL':
        positions=mt5.positions_get(symbol=symbol)
        if positions==None:
            logging.info("No positions on,"+ symbol+", error code={}".format(mt5.last_error()))
        elif len(positions)>0:
            logging.info("Total positions on "+ symbol+" ="+str(len(positions)))
            # display all open positions
            for position in positions:
                if position.profit > 0:
                    logging.info(str(position))
                    lot = position.volume
                    position_id = position.ticket

                    result, close_request  = close_trade(position_id, symbol, lot, deviation)
                    logging.info('result:'+ str(result))
                    logging.info('close_request:'+ str(close_request))
                    
risk = 2/100
account_info_dict = mt5.account_info()._asdict()
Cash = account_info_dict['margin_free']*risk/count_buy
for symbol, action in A.items():                    
    if action == 'BUY' and account_info_dict['margin_free'] > 50:
        
        result, request = open_trade(action, Cash, symbol, sl_points, tp_points, deviation)
        logging.info('request:'+ str(request))
        logging.info('result:'+ str(result))
        
    
    
                    
mt5.shutdown()

volume_min 0.01
volume_max 500.0
volume_min 0.01
volume_max 500.0
volume_min 0.01
volume_max 500.0
Point 1e-05
volume_min 0.01
volume_max 500.0
volume_min 0.01
volume_max 500.0
Point 1e-05
volume_min 0.01
volume_max 1000.0
Point 1e-05
volume_min 0.01
volume_max 1000.0
volume_min 0.1
volume_max 500.0
Point 1e-05
volume_min 0.01
volume_max 1000.0
Point 0.001
volume_min 0.01
volume_max 1000.0
volume_min 0.01
volume_max 1000.0
volume_min 0.01
volume_max 500.0
volume_min 0.01
volume_max 1000.0
volume_min 0.01
volume_max 1000.0
volume_min 0.01
volume_max 1000.0
volume_min 0.01
volume_max 10000.0
volume_min 0.01
volume_max 1000.0
Point 1e-05


True